In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
# Setup paths
RAW_DATA_DIR = Path("data/raw_data")
CLEANED_DATA_DIR = Path("data/cleaned_data")
CLEANED_DATA_DIR.mkdir(exist_ok=True)

#### 3. Lorin 2025 Dataset Processing
#### **Purpose**: Extract training effectiveness and skill assessment data  
#### **Key Features**: Pre/post training accuracy, proficiency levels, learning trajectories

In [9]:

class Lorin2025Processor:
    """
    Processes Lorin et al. (2025) dataset for training effectiveness
    Focuses on skill assessment and learning trajectories
    """
    
    @staticmethod
    def process():
        print("Processing Lorin 2025 Dataset...")
        df = pd.read_csv(RAW_DATA_DIR / "phishing_lorin_2025.csv")
        print(f"   Raw data: {len(df)} rows, {len(df.columns)} columns")
        
        # Feature mapping with detailed documentation
        features = {
            # === IDENTIFIERS ===
            'participant_id': 'ID',  # str: Unique participant identifier
            
            # === BACKGROUND VARIABLES ===
            'email_frequency': 'mailboxfrequency',  # cat: How often checks email - affects exposure risk
            'security_training_prior': 'securitytraining',  # cat: Previous cybersecurity training - moderates susceptibility
            'attention_check_pre': 'attention_check_pre',  # str: Data quality check (should be "Sunfish")
            
            # === PRE-TRAINING SELF-EFFICACY (1-5 scale) ===
            'pre_self_efficacy_1': 'pre_self_1',  # int: "I am confident in my phishing knowledge"
            'pre_self_efficacy_2': 'pre_self_2',  # int: "I am alert to phishing attacks" 
            'pre_self_efficacy_3': 'pre_self_3',  # int: "I am proficient at detecting phishing"
            
            # === PRE-TRAINING KNOWLEDGE TEST (0/1 scoring) ===
            'pre_knowledge_1': 'pre_knowledge_1',  # float: Knowledge item 1 - phishing concepts
            'pre_knowledge_2': 'pre_knowledge_2',  # float: Knowledge item 2 - phishing concepts
            'pre_knowledge_3': 'pre_knowledge_3',  # float: Knowledge item 3 - phishing concepts
            'pre_knowledge_4': 'pre_knowledge_4',  # float: Knowledge item 4 - phishing concepts
            'pre_knowledge_5': 'pre_knowledge_5',  # float: Knowledge item 5 - phishing concepts
            'pre_knowledge_6': 'pre_knowledge_6',  # float: Knowledge item 6 - phishing concepts
            'pre_knowledge_7': 'pre_knowledge_7',  # float: Knowledge item 7 - phishing concepts
            'pre_knowledge_8': 'pre_knowledge_8',  # float: Knowledge item 8 - phishing concepts
            'pre_knowledge_9': 'pre_knowledge_9',  # float: Knowledge item 9 - phishing concepts
            'pre_knowledge_10': 'pre_knowledge_10',  # str: Knowledge item 10 - phishing concepts
            
            # === PRE-TRAINING SA-13 SECURITY ATTITUDE SCALE (1-5 scale) ===
            'pre_sa13_1': 'pre_SA.13_1',  # int: SA-13 item 1 - security engagement
            'pre_sa13_2': 'pre_SA.13_2',  # int: SA-13 item 2 - security attentiveness
            'pre_sa13_3': 'pre_SA.13_3',  # int: SA-13 item 3 - security engagement
            'pre_sa13_4': 'pre_SA.13_4',  # int: SA-13 item 4 - security engagement
            'pre_sa13_5': 'pre_SA.13_5',  # int: SA-13 item 5 - security attentiveness
            'pre_sa13_6': 'pre_SA.13_6',  # int: SA-13 item 6 - security attentiveness
            'pre_sa13_7': 'pre_SA.13_7',  # int: SA-13 item 7 - security resistance (inverted)
            'pre_sa13_8': 'pre_SA.13_8',  # int: SA-13 item 8 - security resistance (inverted)
            'pre_sa13_9': 'pre_SA.13_9',  # int: SA-13 item 9 - security resistance (inverted)
            'pre_sa13_10': 'pre_SA.13_10',  # int: SA-13 item 10 - security resistance (inverted)
            'pre_sa13_11': 'pre_SA.13_11',  # int: SA-13 item 11 - security concern
            'pre_sa13_12': 'pre_SA.13_12',  # int: SA-13 item 12 - security concern
            'pre_sa13_13': 'pre_SA.13_13',  # int: SA-13 item 13 - security concern
            
            # === PRE-TRAINING HAIS-Q KNOWLEDGE ITEMS (1-5 scale) ===
            'pre_haisq_knowledge_1': 'pre_haisq_knowledge_1',  # int: HAIS-Q knowledge - security awareness
            'pre_haisq_knowledge_2': 'pre_haisq_knowledge_2',  # int: HAIS-Q knowledge - security awareness
            'pre_haisq_knowledge_3': 'pre_haisq_knowledge_3',  # int: HAIS-Q knowledge - security awareness
            'pre_haisq_knowledge_4': 'pre_haisq_knowledge_4',  # int: HAIS-Q knowledge - security awareness
            'pre_haisq_knowledge_5': 'pre_haisq_knowledge_5',  # int: HAIS-Q knowledge - security awareness
            'pre_haisq_knowledge_6': 'pre_haisq_knowledge_6',  # int: HAIS-Q knowledge - security awareness
            
            # === PRE-TRAINING EMAIL CLASSIFICATION TASK (0-1 scoring) ===
            'pre_class_1': 'pre_class_1',  # float: Email 1 classification - phishing detection ability
            'pre_class_2': 'pre_class_2',  # float: Email 2 classification - phishing detection ability
            'pre_class_3': 'pre_class_3',  # float: Email 3 classification - phishing detection ability
            'pre_class_4': 'pre_class_4',  # float: Email 4 classification - phishing detection ability
            'pre_class_5': 'pre_class_5',  # float: Email 5 classification - phishing detection ability
            'pre_class_6': 'pre_class_6',  # float: Email 6 classification - phishing detection ability
            'pre_class_7': 'pre_class_7',  # float: Email 7 classification - phishing detection ability
            'pre_class_8': 'pre_class_8',  # float: Email 8 classification - phishing detection ability
            'pre_class_9': 'pre_class_9',  # float: Email 9 classification - phishing detection ability
            'pre_class_10': 'pre_class_10',  # float: Email 10 classification - phishing detection ability
            'pre_class_11': 'pre_class_11',  # float: Email 11 classification - phishing detection ability
            'pre_class_12': 'pre_class_12',  # float: Email 12 classification - phishing detection ability
            'pre_class_13': 'pre_class_13',  # float: Email 13 classification - phishing detection ability
            'pre_class_14': 'pre_class_14',  # float: Email 14 classification - phishing detection ability
            'pre_class_15': 'pre_class_15',  # float: Email 15 classification - phishing detection ability
            'pre_class_16': 'pre_class_16',  # float: Email 16 classification - phishing detection ability
            'pre_class_17': 'pre_class_17',  # float: Email 17 classification - phishing detection ability
            'pre_class_18': 'pre_class_18',  # float: Email 18 classification - phishing detection ability
            'pre_class_19': 'pre_class_19',  # float: Email 19 classification - phishing detection ability
            'pre_class_20': 'pre_class_20',  # float: Email 20 classification - phishing detection ability
            'pre_class_21': 'pre_class_21',  # float: Email 21 classification - phishing detection ability
            'pre_class_22': 'pre_class_22',  # float: Email 22 classification - phishing detection ability
            'pre_class_23': 'pre_class_23',  # float: Email 23 classification - phishing detection ability
            'pre_class_24': 'pre_class_24',  # float: Email 24 classification - phishing detection ability
            
            # === TRAINING ENGAGEMENT CHECKS ===
            'video_1_check': 'video_1_check',  # str: Video 1 attention check - training engagement
            'video_2_check': 'video_2_check',  # str: Video 2 attention check - training engagement
            'video_3_check': 'video_3_check',  # str: Video 3 attention check - training engagement
            
            # === TRAINING FEEDBACK ===
            'feedback_like': 'feedback_like',  # float: What participants liked about training
            'feedback_dislike': 'feedback_dislike',  # float: What participants disliked about training
            'feedback_general': 'feedback_general',  # float: General training feedback
            'feedback_00': 'feedback_00',  # str: Specific feedback item
            'feedback_01': 'feedback_01',  # str: Specific feedback item
            'feedback_02': 'feedback_02',  # str: Specific feedback item
            'feedback_10': 'feedback_10',  # str: Specific feedback item
            'feedback_11': 'feedback_11',  # str: Specific feedback item
            'feedback_12': 'feedback_12',  # str: Specific feedback item
            'feedback_2_videos_1': 'feedback_2_videos_1',  # str: Video feedback
            'feedback_2_videos_2': 'feedback_2_videos_2',  # str: Video feedback
            'feedback_2_videos_3': 'feedback_2_videos_3',  # str: Video feedback
            'feedback_2': 'feedback_2',  # str: General feedback
            'feedback_assign': 'feedback_assign',  # str: Assignment feedback
            'feedback_total': 'feedback_total',  # str: Aggregated training feedback preferences
            
            # === POST-TRAINING EMAIL CLASSIFICATION TASK (0-1 scoring) ===
            'post_class_1': 'post_class_1',  # float: Post: Email 1 classification - training effectiveness
            'post_class_2': 'post_class_2',  # float: Post: Email 2 classification - training effectiveness
            'post_class_3': 'post_class_3',  # float: Post: Email 3 classification - training effectiveness
            'post_class_4': 'post_class_4',  # float: Post: Email 4 classification - training effectiveness
            'post_class_5': 'post_class_5',  # float: Post: Email 5 classification - training effectiveness
            'post_class_6': 'post_class_6',  # float: Post: Email 6 classification - training effectiveness
            'post_class_7': 'post_class_7',  # float: Post: Email 7 classification - training effectiveness
            'post_class_8': 'post_class_8',  # float: Post: Email 8 classification - training effectiveness
            'post_class_9': 'post_class_9',  # float: Post: Email 9 classification - training effectiveness
            'post_class_10': 'post_class_10',  # float: Post: Email 10 classification - training effectiveness
            'post_class_11': 'post_class_11',  # float: Post: Email 11 classification - training effectiveness
            'post_class_12': 'post_class_12',  # float: Post: Email 12 classification - training effectiveness
            'post_class_13': 'post_class_13',  # float: Post: Email 13 classification - training effectiveness
            'post_class_14': 'post_class_14',  # float: Post: Email 14 classification - training effectiveness
            'post_class_15': 'post_class_15',  # float: Post: Email 15 classification - training effectiveness
            'post_class_16': 'post_class_16',  # float: Post: Email 16 classification - training effectiveness
            'post_class_17': 'post_class_17',  # float: Post: Email 17 classification - training effectiveness
            'post_class_18': 'post_class_18',  # float: Post: Email 18 classification - training effectiveness
            'post_class_19': 'post_class_19',  # float: Post: Email 19 classification - training effectiveness
            'post_class_20': 'post_class_20',  # float: Post: Email 20 classification - training effectiveness
            'post_class_21': 'post_class_21',  # float: Post: Email 21 classification - training effectiveness
            'post_class_22': 'post_class_22',  # float: Post: Email 22 classification - training effectiveness
            'post_class_23': 'post_class_23',  # float: Post: Email 23 classification - training effectiveness
            'post_class_24': 'post_class_24',  # float: Post: Email 24 classification - training effectiveness
            
            # === POST-TRAINING MEASURES ===
            'post_attention': 'post_attention',  # int: Post-training attention check - data quality
            'post_knowledge_1': 'post_knowledge_1',  # float: Post: Knowledge item 1 - learning gains
            'post_knowledge_2': 'post_knowledge_2',  # float: Post: Knowledge item 2 - learning gains
            'post_knowledge_3': 'post_knowledge_3',  # float: Post: Knowledge item 3 - learning gains
            'post_knowledge_4': 'post_knowledge_4',  # float: Post: Knowledge item 4 - learning gains
            'post_knowledge_5': 'post_knowledge_5',  # float: Post: Knowledge item 5 - learning gains
            'post_knowledge_6': 'post_knowledge_6',  # float: Post: Knowledge item 6 - learning gains
            'post_knowledge_7': 'post_knowledge_7',  # float: Post: Knowledge item 7 - learning gains
            'post_knowledge_8': 'post_knowledge_8',  # float: Post: Knowledge item 8 - learning gains
            'post_knowledge_9': 'post_knowledge_9',  # float: Post: Knowledge item 9 - learning gains
            'post_knowledge_10': 'post_knowledge_10',  # str: Post: Knowledge item 10 - learning gains
            
            # === HAIS-Q EMAIL/INTERNET SECURITY BEHAVIOR SCALE (1-5 scale) ===
            'haisq_emailinternet_1': 'haisq_emailinternet_1',  # int: HAIS-Q item 1 - email security behavior
            'haisq_emailinternet_2': 'haisq_emailinternet_2',  # int: HAIS-Q item 2 - email security behavior
            'haisq_emailinternet_3': 'haisq_emailinternet_3',  # int: HAIS-Q item 3 - email security behavior
            'haisq_emailinternet_4': 'haisq_emailinternet_4',  # int: HAIS-Q item 4 - email security behavior
            'haisq_emailinternet_5': 'haisq_emailinternet_5',  # int: HAIS-Q item 5 - email security behavior
            'haisq_emailinternet_6': 'haisq_emailinternet_6',  # int: HAIS-Q item 6 - email security behavior
            'haisq_emailinternet_7': 'haisq_emailinternet_7',  # int: HAIS-Q item 7 - internet security behavior
            'haisq_emailinternet_8': 'haisq_emailinternet_8',  # int: HAIS-Q item 8 - internet security behavior
            'haisq_emailinternet_9': 'haisq_emailinternet_9',  # int: HAIS-Q item 9 - internet security behavior
            'haisq_emailinternet_10': 'haisq_emailinternet_10',  # int: HAIS-Q item 10 - internet security behavior
            'haisq_emailinternet_11': 'haisq_emailinternet_11',  # int: HAIS-Q item 11 - internet security behavior
            'haisq_emailinternet_12': 'haisq_emailinternet_12',  # int: HAIS-Q item 12 - internet security behavior
            'haisq_emailinternet_13': 'haisq_emailinternet_13',  # int: HAIS-Q item 13 - internet security behavior
            'haisq_emailinternet_14': 'haisq_emailinternet_14',  # int: HAIS-Q item 14 - internet security behavior
            'haisq_emailinternet_15': 'haisq_emailinternet_15',  # int: HAIS-Q item 15 - internet security behavior
            'haisq_emailinternet_16': 'haisq_emailinternet_16',  # int: HAIS-Q item 16 - internet security behavior
            'haisq_emailinternet_17': 'haisq_emailinternet_17',  # int: HAIS-Q item 17 - internet security behavior
            'haisq_emailinternet_18': 'haisq_emailinternet_18',  # int: HAIS-Q item 18 - internet security behavior
            
            # === DEMOGRAPHICS ===
            'age_category': 'dem_age',  # str: Age range - affects tech familiarity and risk patterns
            'education_level': 'dem_edu',  # int: Education level - affects knowledge and skepticism
            'education_other': 'dem_edu_8_TEXT',  # float: Custom education description
            'it_background': 'dem_it',  # str: IT experience level - major moderator of susceptibility
            'it_background_other': 'dem_it_3_TEXT',  # float: Custom IT background description
            
            # === BIG FIVE PERSONALITY INVENTORY (BFI-10 items, 1-5 scale) ===
            'bfi_1': 'dem_bfi_1',  # int: "I see myself as extraverted, enthusiastic" - Extraversion
            'bfi_2': 'dem_bfi_2',  # int: "I see myself as critical, quarrelsome" - Agreeableness (R)
            'bfi_3': 'dem_bfi_3',  # int: "I see myself as dependable, self-disciplined" - Conscientiousness
            'bfi_4': 'dem_bfi_4',  # int: "I see myself as anxious, easily upset" - Neuroticism
            'bfi_5': 'dem_bfi_5',  # int: "I see myself as open to new experiences" - Openness
            'bfi_6': 'dem_bfi_6',  # int: "I see myself as reserved, quiet" - Extraversion (R)
            'bfi_7': 'dem_bfi_7',  # int: "I see myself as sympathetic, warm" - Agreeableness
            'bfi_8': 'dem_bfi_8',  # int: "I see myself as disorganized, careless" - Conscientiousness (R)
            'bfi_9': 'dem_bfi_9',  # int: "I see myself as calm, emotionally stable" - Neuroticism (R)
            'bfi_10': 'dem_bfi_10',  # int: "I see myself as conventional, uncreative" - Openness (R)
            
            # === POST-TRAINING SELF-EFFICACY (1-5 scale) ===
            'post_self_1': 'post_self_1',  # int: Post: "I am confident in my phishing knowledge"
            'post_self_2': 'post_self_2',  # int: Post: "I am alert to phishing attacks"
            'post_self_3': 'post_self_3',  # int: Post: "I am proficient at detecting phishing"
            
            # === POST-TRAINING SA-13 SECURITY ATTITUDES (1-5 scale) ===
            'post_sa13_1': 'post_sa.13_1',  # int: Post SA-13 item 1 - security engagement
            'post_sa13_2': 'post_sa.13_2',  # int: Post SA-13 item 2 - security attentiveness
            'post_sa13_3': 'post_sa.13_3',  # int: Post SA-13 item 3 - security engagement
            'post_sa13_4': 'post_sa.13_4',  # int: Post SA-13 item 4 - security engagement
            'post_sa13_5': 'post_sa.13_5',  # int: Post SA-13 item 5 - security attentiveness
            'post_sa13_6': 'post_sa.13_6',  # int: Post SA-13 item 6 - security attentiveness
            'post_sa13_7': 'post_sa.13_7',  # int: Post SA-13 item 7 - security resistance (R)
            'post_sa13_8': 'post_sa.13_8',  # int: Post SA-13 item 8 - security resistance (R)
            'post_sa13_9': 'post_sa.13_9',  # int: Post SA-13 item 9 - security resistance (R)
            'post_sa13_10': 'post_sa.13_10',  # int: Post SA-13 item 10 - security resistance (R)
            'post_sa13_11': 'post_sa.13_11',  # int: Post SA-13 item 11 - security concern
            'post_sa13_12': 'post_sa.13_12',  # int: Post SA-13 item 12 - security concern
            'post_sa13_13': 'post_sa.13_13',  # int: Post SA-13 item 13 - security concern
            
            # === EXPERIMENTAL CONDITIONS & GROUPS ===
            'training_condition': 'condition',  # int: 0=Personalised, 1=Random, 2=Education-only - KEY PREDICTOR
            'proficiency_group': 'group',  # int: 0=Low, 1=Medium, 2=High proficiency - KEY MODERATOR
            'prof_group': 'prof_group',  # int: Final proficiency group assignment
            'train_group': 'train_group',  # str: Actual training group received (text)
            'assignment_fit': 'assignment_fit',  # str: How well training matched proficiency level
            
            # === PRE-TRAINING COMPOSITE SCORES ===
            'pre_sa13_engagement': 'pre.sa13_engagement',  # float: Pre: SA-13 engagement subscale
            'pre_sa13_attentiveness': 'pre.sa13_attentiveness',  # float: Pre: SA-13 attentiveness subscale
            'pre_sa13_resistance': 'pre.sa13_resistance',  # float: Pre: SA-13 resistance subscale (higher = more resistant)
            'pre_sa13_concernedness': 'pre.sa13_concernedness',  # float: Pre: SA-13 concern subscale
            'pre_sa13_total': 'pre.sa13_total',  # float: Pre: SA-13 total security attitude score
            
            # === POST-TRAINING COMPOSITE SCORES ===
            'post_sa13_engagement': 'post.sa13_engagement',  # float: Post: SA-13 engagement subscale
            'post_sa13_attentiveness': 'post.sa13_attentiveness',  # float: Post: SA-13 attentiveness subscale
            'post_sa13_resistance': 'post.sa13_resistance',  # float: Post: SA-13 resistance subscale
            'post_sa13_concernedness': 'post.sa13_concernedness',  # float: Post: SA-13 concern subscale
            'post_sa13_total': 'post.sa13_total',  # float: Post: SA-13 total security attitude score
            
            # === HAIS-Q COMPOSITE SCORES ===
            'pre_haisq_email_knowledge': 'pre.haisq_email_knowledge',  # int: Pre: Email security knowledge
            'pre_haisq_internet_knowledge': 'pre.haisq_internet_knowledge',  # int: Pre: Internet security knowledge
            'post_haisq_email_knowledge': 'post.haisq_email_knowledge',  # int: Post: Email security knowledge
            'post_haisq_email_attitude': 'post.haisq_email_attitude',  # int: Post: Email security attitude
            'post_haisq_email_behaviour': 'post.haisq_email_behaviour',  # int: Post: Email security behavior
            'post_haisq_internet_knowledge': 'post.haisq_internet_knowledge',  # int: Post: Internet security knowledge
            'post_haisq_internet_attitude': 'post.haisq_internet_attitude',  # int: Post: Internet security attitude
            'post_haisq_internet_behaviour': 'post.haisq_internet_behaviour',  # int: Post: Internet security behavior
            'post_haisq_total': 'post.haisq_emailInternet_total',  # int: Post: Combined email/internet security score
            
            # === BIG FIVE COMPOSITE SCORES ===
            'bfi_extraversion': 'bfi_extraversion',  # float: Extraversion score - social engineering vulnerability
            'bfi_agreeableness': 'bfi_agreeableness',  # float: Agreeableness score - trust/cooperation tendency
            'bfi_conscientiousness': 'bfi_conscientiousness',  # float: Conscientiousness score - careful/organized behavior
            'bfi_neuroticism': 'bfi_neuroticism',  # float: Neuroticism score - anxiety/stress response
            'bfi_openness': 'bfi_openness',  # float: Openness score - curiosity/exploration tendency
            
            # === PRIMARY PERFORMANCE METRICS - CRITICAL FOR PERSONAS ===
            'class_phish_accuracy_pre': 'class_phish_accuracy_pre',  # float: Pre: Phishing email detection accuracy (0-1)
            'class_nophish_accuracy_pre': 'class_nophish_accuracy_pre',  # float: Pre: Legitimate email accuracy (0-1)
            'class_phish_accuracy_post': 'class_phish_accuracy_post',  # float: Post: Phishing email detection accuracy (0-1)
            'class_nophish_accuracy_post': 'class_nophish_accuracy_post',  # float: Post: Legitimate email accuracy (0-1)
            'knowledge_total_pre': 'knowledge_total_pre',  # float: Pre: Total knowledge test score (0-10)
            'knowledge_total_post': 'knowledge_total_post',  # float: Post: Total knowledge test score (0-10)
            'proficiency_pre': 'proficiency_pre',  # float: Pre: Overall proficiency score - KEY BASELINE
            'proficiency_post': 'proficiency_post',  # float: Post: Overall proficiency score - KEY OUTCOME
            
            # === QUALITY CONTROL MEASURES ===
            'participant_score': 'participant_score',  # float: Final calculated proficiency score
            'reports': 'reports',  # str: Security incident reports - real-world experience
            'outlier_flag': 'outlier'  # float: Statistical outlier identification flag
        }
        
        # Create cleaned dataframe
        cleaned = pd.DataFrame()
        for new_col, old_col in features.items():
            if old_col in df.columns:
                cleaned[new_col] = df[old_col]
            else:
                print(f"Column '{old_col}' not found, setting '{new_col}' to NaN")
                cleaned[new_col] = np.nan
        
        # Remove timestamp columns (all timestamp columns)
        timestamp_cols = [col for col in df.columns if 'timestamp' in col.lower() or 'recorded' in col.lower()]
        if timestamp_cols:
            print(f"Removed {len(timestamp_cols)} timestamp columns")
        
        # Convert boolean knowledge test columns
        knowledge_cols = [col for col in cleaned.columns if 'knowledge_q' in col]
        for col in knowledge_cols:
            if col in cleaned.columns:
                cleaned[col] = pd.to_numeric(cleaned[col], errors='coerce').fillna(0).astype(int)
        
        print(f"Processed: {len(cleaned)} rows, {len(cleaned.columns)} features")
        return cleaned


In [10]:
lorin2025_data = Lorin2025Processor.process()
lorin2025_data.to_csv(CLEANED_DATA_DIR / "lorin_2025_cleaned.csv", index=False)
print(f"Saved to: {CLEANED_DATA_DIR / 'lorin_2025_cleaned.csv'}")

Processing Lorin 2025 Dataset...
   Raw data: 342 rows, 267 columns
Removed 7 timestamp columns
Processed: 342 rows, 202 features
Saved to: data/cleaned_data/lorin_2025_cleaned.csv
